# Poisoning Attack in Federated Learning Experiment 1.1

Purpose: 
- What is the performance loss induced through the different poisoning attack? 
- What is the performance of the global model without poisoning?


## Google Colab

In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

In [3]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Libraries

In [4]:
from federated_learning.utils import SHAPUtil, experiment_util
from federated_learning import ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server
from datetime import datetime

## Additional Functions

In [5]:
class Experiment():
    def __init__(self, num_p_clients, from_label, to_label, natural_label ):
        self.num_p_clients = num_p_clients
        self.from_label = from_label
        self.to_label = to_label
        self.neutral_label = natural_label
        self.accuracy = []
        self.recall = {}
        self.initialize_recall()

    def initialize_recall(self):
        self.recall[self.from_label] = []
        self.recall[self.to_label] = []
        self.recall[self.neutral_label] = []
        

In [ ]:
experiments = []

In [6]:
def add_round_to_experiment(experiment, recall, accuracy, from_label, to_label, neutral_label): 
    experiment.accuracy.append(accuracy)
    experiment.recall[from_label].append(recall[from_label])
    experiment.recall[to_label].append(recall[to_label])
    experiment.recall[neutral_label].append(recall[neutral_label])

## MNIST(5,4)

In [13]:
from federated_learning.nets import MNISTCNN
from federated_learning.dataset import MNISTDataset
import os
config = Configuration()
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = MNISTDataset
config.MODELNAME = config.MNIST_NAME
config.NETWORK = MNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [14]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/data/fmnist/MNIST/raw/train-images-idx3-ubyte.gz to /content/data/fmnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/data/fmnist/MNIST/raw/train-labels-idx1-ubyte.gz to /content/data/fmnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/data/fmnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/data/fmnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/data/fmnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/data/fmnist/MNIST/raw

MNIST training data loaded.
MNIST test data loaded.


In [15]:
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)

Create 200 clients with dataset of size 300


In [ ]:
import numpy as np
import copy
for num_p_clients in [0, 10, 25, 50,75, 100, 200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    client_plane.reset_poisoning_attack()
    config.POISONED_CLIENTS = num_p_clients
    experiment = Experiment(num_p_clients, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    server.test()
    recall, precision, accuracy = server.analize_test()
    add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    for i in range(200):
        experiment_util.set_rounds(client_plane, server, i+1)
        experiment_util.run_round(client_plane, server, i+1)
        if (i+1)%10 == 0:
            server.test()
            recall, precision, accuracy = server.analize_test()
            add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
            print("Round {} finished".format(i+1))
    print(experiment.accuracy)
    print(experiment.recall[config.FROM_LABEL])
    print(experiment.recall[config.TO_LABEL])
    print(experiment.recall[natural_label])
    experiments.append(copy.deepcopy(experiment))

In [ ]:
for e in experiments: 
    print(e.accuracyracy)
    print(e.recall[config.FROM_LABEL])
    print(e.recall[config.TO_LABEL])
    print(e.recall[neutral_label])

# Fashion MNIST(5,4)

In [10]:
from federated_learning.nets import FMNISTCNN
from federated_learning.dataset import FMNISTDataset
import os
config = Configuration()
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = FMNISTDataset
config.MODELNAME = config.FMNIST_NAME
config.NETWORK = FMNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.dataset = "FMNIST"
neutral_label = 2

In [11]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
experiments_fmnist = []

FashionMnist training data loaded.
FashionMnist training data loaded.
Create 200 clients with dataset of size 300


In [12]:
import numpy as np
import copy
for num_p_clients in [0, 10, 25, 50, 75, 100, 200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    client_plane.reset_poisoning_attack()
    config.POISONED_CLIENTS = num_p_clients
    experiment = Experiment(num_p_clients, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    server.test()
    recall, precision, accuracy = server.analize_test()
    add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    for i in range(200):
        experiment_util.set_rounds(client_plane, server, i+1)
        experiment_util.run_round(client_plane, server, i+1)
        if (i+1)%10 == 0:
            server.test()
            recall, precision, accuracy = server.analize_test()
            add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
            print("Round {} finished".format(i+1))
    print(experiment.accuracy)
    print(experiment.recall[config.FROM_LABEL])
    print(experiment.recall[config.TO_LABEL])
    print(experiment.recall[neutral_label])
    experiments_fmnist.append(copy.deepcopy(experiment))

Load default model successfully
No poisoning due to 0. poisoned clients

Test set: Average loss: 0.0023, Accuracy: 1252/10000 (13%)


Test set: Average loss: 0.0005, Accuracy: 8182/10000 (82%)

Round 10 finished

Test set: Average loss: 0.0004, Accuracy: 8508/10000 (85%)

Round 20 finished

Test set: Average loss: 0.0004, Accuracy: 8451/10000 (85%)

Round 30 finished

Test set: Average loss: 0.0004, Accuracy: 8635/10000 (86%)

Round 40 finished
Model aggregation in round 50 was successful

Test set: Average loss: 0.0004, Accuracy: 8595/10000 (86%)

Round 50 finished

Test set: Average loss: 0.0004, Accuracy: 8745/10000 (87%)

Round 60 finished

Test set: Average loss: 0.0003, Accuracy: 8793/10000 (88%)

Round 70 finished

Test set: Average loss: 0.0003, Accuracy: 8812/10000 (88%)

Round 80 finished

Test set: Average loss: 0.0003, Accuracy: 8813/10000 (88%)

Round 90 finished
Model aggregation in round 100 was successful
Train Epoch: 100 [0/300 (0%)]	Loss: 0.220859
Train Epoch: 100 [10

In [19]:
for e in experiments_fmnist: 
    print(e.accuracy)
    print(e.recall[config.FROM_LABEL])
    print(e.recall[config.TO_LABEL])
    print(e.recall[neutral_label])

[0.1252, 0.8182, 0.8508, 0.8451, 0.8635, 0.8595, 0.8745, 0.8793, 0.8812, 0.8813, 0.8844, 0.887, 0.8901, 0.8917, 0.8943, 0.8924, 0.894, 0.8955, 0.8947, 0.8984, 0.8978]
[tensor(0.), tensor(0.8550), tensor(0.9480), tensor(0.9860), tensor(0.9350), tensor(0.9350), tensor(0.9760), tensor(0.9590), tensor(0.9640), tensor(0.9550), tensor(0.9640), tensor(0.9660), tensor(0.9630), tensor(0.9560), tensor(0.9670), tensor(0.9570), tensor(0.9770), tensor(0.9830), tensor(0.9620), tensor(0.9840), tensor(0.9530)]
[tensor(0.), tensor(0.6480), tensor(0.7150), tensor(0.8490), tensor(0.8920), tensor(0.6770), tensor(0.7790), tensor(0.8080), tensor(0.7760), tensor(0.8730), tensor(0.7660), tensor(0.7710), tensor(0.8840), tensor(0.8660), tensor(0.9060), tensor(0.7940), tensor(0.8460), tensor(0.8690), tensor(0.7940), tensor(0.8340), tensor(0.8900)]
[tensor(0.0280), tensor(0.7660), tensor(0.7240), tensor(0.6540), tensor(0.7310), tensor(0.8890), tensor(0.7840), tensor(0.7270), tensor(0.8770), tensor(0.7250), tensor